# Export monthly temperatures
- This script is used to export temperatures from simulations;
- Simulations: CNTL, TRAF at UK-Manchester. 

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
archive = f'{home_path}0_urban_traffic/archive'
start_date = '2012-01-01T00:00:00'
end_date = '2012-12-31T23:00:00'
case_list = ['cntl', 'traffic'] 
GRIDNAME_single = 'UK-MCR'  

In [3]:
var_list_hourly = ['TSA_U', 'TBUILD', 'TG_U']
df_hourly = pd.DataFrame()
for case in case_list:
    ds_case = xr.open_dataset(f'{archive}/{GRIDNAME_single}_{case}/lnd/hist/{GRIDNAME_single}_{case}.clm2.h0.2022-01-01-03600.nc')
    for var in var_list_hourly : 
        ds_case_var = ds_case[var] - 273.15
        df_case_var=ds_case_var.to_dataframe().reset_index()[['time', var]]
        df_case_var.rename(columns={var: f'{var}_{case}'}, inplace=True)
        df_case_var['time'] = df_case_var['time'].dt.round('min').dt.ceil('min')
        df_case_var['time'] = pd.to_datetime(df_case_var['time'])
        if df_hourly.empty:
            df_hourly = df_case_var
        else:
            df_hourly = pd.merge(df_hourly, df_case_var, how='outer', on='time') 
df_hourly['time'] = df_hourly['time'] - pd.Timedelta(hours=1)
df_hourly['outdoor_indoor_cntl'] = df_hourly['TSA_U_cntl'] - df_hourly['TBUILD_cntl']
df_hourly['outdoor_indoor_traffic'] = df_hourly['TSA_U_traffic'] - df_hourly['TBUILD_traffic']
df_hourly['date'] = df_hourly['time'].dt.date
df_hourly2daily = df_hourly.groupby(['date']).mean().reset_index()          
df_hourly2daily.drop(columns=['time'], inplace=True)
df_hourly2daily['month'] = pd.to_datetime(df_hourly2daily['date']).dt.month  
df_hourly2daily.drop(columns=['date'], inplace=True)
df_monthly = df_hourly2daily.groupby(['month']).mean().reset_index()
df_monthly_5 = df_hourly2daily.groupby(['month']).quantile(0.05).reset_index()
df_monthly_95 = df_hourly2daily.groupby(['month']).quantile(0.95).reset_index()
df_monthly.head()

,month,TSA_U_cntl,TBUILD_cntl,TG_U_cntl,TSA_U_traffic,TBUILD_traffic,TG_U_traffic,outdoor_indoor_cntl,outdoor_indoor_traffic
0,1,5.509286,16.950012,4.972656,5.860288,16.950012,5.471256,-11.440726,-11.089724
1,2,7.503757,16.950012,6.798150,7.715301,16.950012,7.145034,-9.446256,-9.234712
2,3,8.908651,16.950012,9.199816,9.125525,16.950012,9.586072,-8.041361,-7.824487
3,4,10.789529,16.963062,11.787928,10.950852,16.967630,12.126372,-6.173533,-6.016779
4,5,13.954983,17.024183,14.845215,14.105767,17.039860,15.150421,-3.069202,-2.934093


In [4]:
var_list_daily = ['TGMNAV_U', 'TGMXAV_U', 'TREFMXAV_U', 'TREFMNAV_U']
df_daily = pd.DataFrame()
for case in case_list:
    ds_case_daily = xr.open_dataset(f'{archive}/{GRIDNAME_single}_{case}/lnd/hist/{GRIDNAME_single}_{case}.clm2.h1.2022-01-02-00000.nc')
    for var in var_list_daily:
        ds_case_var = ds_case_daily[var] - 273.15
        df_case_var=ds_case_var.to_dataframe().reset_index()[['time', var]]
        df_case_var.rename(columns={var: f'{var}_{case}'}, inplace=True)
        df_case_var['time'] = df_case_var['time'].dt.round('min').dt.ceil('min')
        df_case_var['time'] = pd.to_datetime(df_case_var['time'])
        if df_daily.empty:
            df_daily = df_case_var
        else:
            df_daily = pd.merge(df_daily, df_case_var, how='outer', on='time') 
df_daily['month'] = df_daily['time'].dt.month 
df_daily['TG_daynight_cntl'] = df_daily['TGMXAV_U_cntl'] - df_daily['TGMNAV_U_cntl']
df_daily['TG_daynight_traffic'] = df_daily['TGMXAV_U_traffic'] - df_daily['TGMNAV_U_traffic']
df_daily['TSA_daynight_cntl'] = df_daily['TREFMXAV_U_cntl'] - df_daily['TREFMNAV_U_cntl']
df_daily['TSA_daynight_traffic'] = df_daily['TREFMXAV_U_traffic'] - df_daily['TREFMNAV_U_traffic']
df_daily_delta = df_daily.groupby(['month']).mean().reset_index()
df_daily_delta_5 = df_daily.groupby(['month']).quantile(0.05).reset_index()
df_daily_delta_95 = df_daily.groupby(['month']).quantile(0.95).reset_index()
df_daily_delta.drop(columns=['time'], inplace=True)
df_daily_delta_5.drop(columns=['time'], inplace=True)
df_daily_delta_95.drop(columns=['time'], inplace=True)
df_daily_delta.head()          

,month,TGMNAV_U_cntl,TGMXAV_U_cntl,TREFMXAV_U_cntl,TREFMNAV_U_cntl,TGMNAV_U_traffic,TGMXAV_U_traffic,TREFMXAV_U_traffic,TREFMNAV_U_traffic,TG_daynight_cntl,TG_daynight_traffic,TSA_daynight_cntl,TSA_daynight_traffic
0,1,2.300158,8.160870,8.153376,2.968301,2.742287,8.716318,8.472425,3.332132,5.860712,5.974031,5.185074,5.140294
1,2,3.895779,10.155286,9.991365,4.928285,4.224247,10.518909,10.168480,5.261468,6.259506,6.294662,5.063081,4.907012
2,3,3.381514,17.134909,13.823826,4.316633,3.693201,17.516611,13.929020,4.602558,13.753395,13.823411,9.507194,9.326463
3,4,5.021863,20.800591,15.709046,5.932935,5.273626,21.153011,15.783583,6.185023,15.778728,15.879386,9.776112,9.598559
4,5,9.021338,22.255276,17.957506,9.742521,9.287904,22.569622,18.026752,10.004151,13.233939,13.281717,8.214985,8.022601


In [5]:
df_all = pd.merge(df_monthly, df_daily_delta, how='outer', on='month')
df_all_5 = pd.merge(df_monthly_5, df_daily_delta_5, how='outer', on='month')
df_all_95 = pd.merge(df_monthly_95, df_daily_delta_95, how='outer', on='month')
df_all.to_csv('./data_for_figure/monthly_mean.csv', index=False)
df_all_5.to_csv('./data_for_figure/monthly_5.csv', index=False)
df_all_95.to_csv('./data_for_figure/monthly_95.csv', index=False)
df_all.head()

,month,TSA_U_cntl,TBUILD_cntl,TG_U_cntl,TSA_U_traffic,TBUILD_traffic,TG_U_traffic,outdoor_indoor_cntl,outdoor_indoor_traffic,TGMNAV_U_cntl,...,TREFMXAV_U_cntl,TREFMNAV_U_cntl,TGMNAV_U_traffic,TGMXAV_U_traffic,TREFMXAV_U_traffic,TREFMNAV_U_traffic,TG_daynight_cntl,TG_daynight_traffic,TSA_daynight_cntl,TSA_daynight_traffic
0,1,5.509286,16.950012,4.972656,5.860288,16.950012,5.471256,-11.440726,-11.089724,2.300158,...,8.153376,2.968301,2.742287,8.716318,8.472425,3.332132,5.860712,5.974031,5.185074,5.140294
1,2,7.503757,16.950012,6.798150,7.715301,16.950012,7.145034,-9.446256,-9.234712,3.895779,...,9.991365,4.928285,4.224247,10.518909,10.168480,5.261468,6.259506,6.294662,5.063081,4.907012
2,3,8.908651,16.950012,9.199816,9.125525,16.950012,9.586072,-8.041361,-7.824487,3.381514,...,13.823826,4.316633,3.693201,17.516611,13.929020,4.602558,13.753395,13.823411,9.507194,9.326463
3,4,10.789529,16.963062,11.787928,10.950852,16.967630,12.126372,-6.173533,-6.016779,5.021863,...,15.709046,5.932935,5.273626,21.153011,15.783583,6.185023,15.778728,15.879386,9.776112,9.598559
4,5,13.954983,17.024183,14.845215,14.105767,17.039860,15.150421,-3.069202,-2.934093,9.021338,...,17.957506,9.742521,9.287904,22.569622,18.026752,10.004151,13.233939,13.281717,8.214985,8.022601
